## Microsoft Agent Framework
[Microsoft Agent Framework](https://github.com/microsoft/agent-framework) is an open-source development kit for building AI agents and multi-agent workflows for .NET and Python. It brings together and extends ideas from [Semantic Kernel](https://github.com/microsoft/semantic-kernel) and [AutoGen](https://github.com/microsoft/autogen) projects, combining their strengths while adding new capabilities.
Built by the same teams, it is the unified foundation for building AI agents going forward.

Agent Framework offers two primary categories of capabilities:

- [AI agents](https://learn.microsoft.com/en-us/agent-framework/overview/agent-framework-overview#ai-agents): Individual agents that use LLMs to process user inputs, call tools and MCP servers to perform actions, and generate responses. Agents support model providers including Azure OpenAI, OpenAI, and Azure AI.
- [Workflows](https://learn.microsoft.com/en-us/agent-framework/overview/agent-framework-overview#workflows): Graph-based workflows that connect multiple agents and functions to perform complex, multi-step tasks. Workflows support type-based routing, nesting, checkpointing, and request/response patterns for human-in-the-loop scenarios.

### Agents
All agents are derived from a common base class, `AIAgent`, which provides a consistent interface for all agent types. 

Agent Framework supports many different types of agents. This tutorial shows you how to create and run an agent with Agent Framework based on the Azure OpenAI Chat Completion service, but all other agent types are run in the same way.


These agents support a wide range of functionality out of the box:

1. Function calling
2. Multi-turn conversations with local chat history management or service provided chat history management
3. Custom service provided tools (e.g. MCP, Code Execution)
4. Structured output
5. Streaming responses


For more information on other agent types and how to construct them, see the [MAF Agent Types](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/agent-types/?pivots=programming-language-python)

### Our first Agent

Now, let's build an single agent to get started. First, create a chat client for communicating with Azure OpenAI:

```
AZURE_OPENAI_ENDPOINT - The endpoint it should talk to by default
AZURE_OPENAI_API_KEY - The API Key it should use
AZURE_OPENAI_API_VERSION - Inference API version it should use per default
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME - Model deployment name it should use per default
AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME - Embedding deployment name it should use per default
```

In [ ]:
import os
from dotenv import load_dotenv
from agent_framework.azure import AzureOpenAIChatClient

# Load environment variables
load_dotenv()

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

# Create the client using API key
client = AzureOpenAIChatClient(
    endpoint=endpoint,
    api_key=api_key,
    api_version=api_version,
    deployment_name=deployment
)

To create an agent, simply construct a `ChatAgent` using the chat client implementation of your choice:

```python
simple_agent = ChatAgent(
	chat_client = AzureOpenAIChatClient(),
	instructions = "You are a helpful assistant"
)
```


Below is an example of using a convenience method `create_agent()` for creating a ChatAgent tied to our Azure OpenAI Chat Completion client:

In [ ]:
simple_agent = client.create_agent(
    instructions="You are an AI assistant that helps users with their questions.",
    name="AI Assistant"
)

### Running the agent 
To run the agent, call the `run` method on the agent instance, providing the user input. The agent will return a response object, and accessing the `.text` property provides the text result from the agent.

To run the agent with streaming, call the `run_stream` method on the agent instance, providing the user input.

The agent will stream a list of update objects, and accessing the `.text` property on each update object provides the part of the text result contained in that update.

Python agents support passing keyword arguments to customize each run. The specific options available depend on the agent type, but `ChatAgent` supports many chat client parameters that can be passed to both `run` and `run_stream` methods. Common options include `max_tokens`, `temperature`, `model`, `tools`, `response_format`.

Please note that we do not have an chat history, so every message we send to the agent is treated as a new conversation.

In [ ]:
# Regular run
response = await simple_agent.run("What's the capital of Latvia?")
print("Full response:", response.text)

# Streaming run
print("\nStreaming response:")
async for chunk in simple_agent.run_stream("Explain why having a capital for a country is important in two short sentences.", 
        max_tokens=30):
    # Each chunk is a partial piece of the model's output
    if chunk.text:
            print(chunk.text, end="", flush=True)
print()

### Threading and Multi-Turn Conversations

So far, our agents are stateless and do not maintain any state internally between calls. If you ask them a follow up question, they would have no reference to the prior conversation.

To have a multi-turn conversation with an agent, you need to create an object to hold the conversation state and pass this object to the agent when running it.

An `AgentThread` maintains the conversation state and message history for an agent interaction. It can either use a service-managed thread (via service_thread_id) or a local message store (via message_store), but not both.

To create the conversation state object, call the `get_new_thread()` method on the agent instance. In case you do not provide an `AgentThread`, the agent will create a throwaway underlying thread which will be discarded after the run.

You can then pass this thread object to the `run` or `run_stream` methods on the agent instance, along with the user input.


In [ ]:
agent = client.create_agent(
    instructions="You are an AI assistant that helps users with their questions.",
    name="AI Assistant"
)

# create a thread for the agent instance:
thread = agent.get_new_thread()

user_messages = [
    "Hello!",
    "Which country has Paris as the capital?",
    "What are its neighbouring countries? Give a short one-liner list, please."
]

# Loop through user messages and maintain context
for user_message in user_messages:
    print("*** User:", user_message)
    
    # get response from the agent, passing the same thread
    response = await agent.run(user_message, thread=thread)
    print("*** Agent:", response.text)

print("-" * 25)

# Print the final conversation history from the thread
messages = await thread.message_store.list_messages()
for msg in messages:
    print(f"[{msg.role}] {msg.text}")


Learn more about threading, custom message stores and serialization [here](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/multi-turn-conversation?pivots=programming-language-python) and [here](https://learn.microsoft.com/en-us/agent-framework/tutorials/agents/persisted-conversation?pivots=programming-language-python)

### Single agent with multiple conversations

It is possible to have multiple, independent conversations with the same agent instance by creating multiple `AgentThread` objects. These threads can then be used to maintain separate conversation states for each conversation. The conversations will be fully independent of each other, since the agent does not maintain any state internally.


In [ ]:
thread1 = agent.get_new_thread()
thread2 = agent.get_new_thread()

# First messages for each thread
result1 = await agent.run("Suggest 1 key attraction for Paris trip. Keep it brief.", thread=thread1)
print("Thread 1:", result1.text)

result2 = await agent.run("Suggest 1 key attraction for Tokyo trip. Keep it brief.", thread=thread2)
print("Thread 2:", result2.text)

# Continue each conversation independently
result3 = await agent.run(
    "Now suggest one morning activity. One short sentence.",
    thread=thread1
)
print("Thread 1:", result3.text)

result4 = await agent.run(
    "Now suggest one morning activity. One short sentence.",
    thread=thread2
)
print("Thread 2:", result4.text)

<details>
    <summary> See the solution</summary>

```python
thread1 = agent.get_new_thread()
thread2 = agent.get_new_thread()

# First messages for each thread
result1 = await agent.run("Suggest 1 key attraction for Paris trip. Keep it brief.", thread=thread1)
print("Thread 1:", result1.text)

result2 = await agent.run("Suggest 1 key attraction for Tokyo trip. Keep it brief.", thread=thread2)
print("Thread 2:", result2.text)

# Continue each conversation independently
result3 = await agent.run(
    "Now suggest one morning activity. One short sentence.",
    thread=thread1
)
print("Thread 1:", result3.text)

result4 = await agent.run(
    "Now suggest one morning activity. One short sentence.",
    thread=thread2
)
print("Thread 2:", result4.text)
```
</details>

### Agent Function Tools
Tooling support may vary considerably between different agent types. Some agents may allow developers to customize the agent at construction time by providing external function tools or by choosing to activate specific built-in tools that are supported by the agent. 

The `ChatAgent` is an agent class that can be used to build agentic capabilities on top of any inference service. It comes with support for:

1. Using your own function tools with the agent
2. Using built-in tools that the underlying service may support
3. Using hosted tools like web search and MCP (Model Context Protocol) servers

Let's try to provide function tools during agent construction:

In [ ]:
from typing import Annotated
from agent_framework import ChatAgent
from agent_framework.azure import AzureOpenAIChatClient
from datetime import datetime

def get_time() -> str:
    return datetime.now().strftime("%H:%M:%S")

# Sample function tool
def get_weather(
    location: Annotated[str, "The location to get the weather for."],
) -> str:
    """Get the weather for a given location."""
    return f"The weather in {location} is cloudy with a high of 15°C."

# Agent with agent-level tool available  for all runs
agent = ChatAgent(
    chat_client=AzureOpenAIChatClient(),
    instructions="You are a helpful assistant",
    tools=[get_time] 
)

# This run has access to both get_time (agent-level) and get_weather (run-level)
result = await agent.run(
    "What's the weather and time in New York?",
    tools=[get_weather]  # Additional tool for this run
)

print(result.text)

You can also use the `ai_function` decorator to explicitly define function name and description for LLM readability and behaviour:
```python
from agent_framework import ai_function

@ai_function(name="weather_tool", description="Retrieves weather information")
# your function here:
def get_weather(...)

```

If you don't use a decorator, the framework will use function name and docstring by default.

You can also create a class that contains **multiple function tools** as methods.

 In the following example, let's try to create an agent that works as a note taking assistant: 

In [ ]:
from typing import Annotated, List, Tuple
from datetime import datetime

class NotesTools:
    def __init__(self):
        self.notes = []  # Store notes as (timestamp, note)

    def list_notes(self) -> Annotated[List[Tuple[str, str]], "Returns all notes as (timestamp, note)."]:
        """Return all notes with their timestamps."""
        if not self.notes:
            return "No notes available."
        return self.notes

    def write_note(
        self,
        note: Annotated[str, "The note message to save."],
    ) -> str:
        """Save a note with the current timestamp."""
        timestamp = datetime.now().isoformat()
        self.notes.append((timestamp, note))
        return f"Note added at {timestamp}."

    
# create tools instance
tools = NotesTools()

# create agent with tools
agent = client.create_agent(
    instructions="You are an assistant that logs, stores notes and shows them. Everything i send is a note i want you to log in my notebook.",
    tools=[tools.write_note, tools.list_notes]
)

# create a thread for multi-turn conversation
thread = agent.get_new_thread()

user_messages = [
    "Find new soup recipes.",
    "Find new project opportunities at university.",
    "Don't forget to call Jess to say happy birthday.",
    "Can you list all my notes?"
]

for m in user_messages:
    print("*** User:", m)
    response = await agent.run(m, thread=thread)
    print("*** Agent:", response.text)

<details>
    <summary>See the solution</summary>

```python
from typing import Annotated, List, Tuple
from datetime import datetime

class NotesTools:
    def __init__(self):
        self.notes = []  # Store notes as (timestamp, note)

    def list_notes(self) -> Annotated[List[Tuple[str, str]], "Returns all notes as (timestamp, note)."]:
        """Return all notes with their timestamps."""
        if not self.notes:
            return "No notes available."
        return self.notes

    def write_note(
        self,
        note: Annotated[str, "The note message to save."],
    ) -> str:
        """Save a note with the current timestamp."""
        timestamp = datetime.now().isoformat()
        self.notes.append((timestamp, note))
        return f"Note added at {timestamp}."

    
# create tools instance
tools = NotesTools()

# create agent with tools
agent = client.create_agent(
    instructions="You are an assistant that logs, stores notes and shows them. Everything i send is a note i want you to log in my notebook.",
    tools=[tools.write_note, tools.list_notes]
)

# create a thread for multi-turn conversation
thread = agent.get_new_thread()

user_messages = [
    "Find new soup recipes.",
    "Find new project opportunities at university.",
    "Don't forget to call Jess to say happy birthday.",
    "Can you list all my notes?"
]

for m in user_messages:
    print("*** User:", m)
    response = await agent.run(m, thread=thread)
    print("*** Agent:", response.text)

```    
</details>

## Exercise - Implement an Agent with Function Tools

In this exercise, you'll create an agent that manages tasks with priorities and deadlines. This will help you understand how to create and expose function tools to an agent and run a multi-turn conversation.

#### Task:
1. Create a `TaskManagerTools` class with the following functions:
    - `add_task(name, priority, deadline)` - Adds a new task with a description, priority (high, medium, low), and deadline in DD-MM format.
    - `list_tasks()` - Returns all tasks with their details.
    - `filter_by_priority(priority)` - Returns tasks that match the given priority.

2. Annotate parameters with `Annotated` to provide descriptions for better LLM understanding.
3. Create an agent with clear instructions and register function tools
4. Test the agent with a multi-turn conversation

In [ ]:
from typing import Annotated, List, Dict
from datetime import datetime


class TaskManagerTools:
    def __init__(self):
        self.tasks: List[Dict[str, str]] = []  # expected list of dicts, each task: {"name": str, "priority": str, "deadline": str}

    def add_task(
        self,
        name: Annotated[str, "The task description."],
        priority: Annotated[str, "Priority level: high, medium, low."],
        deadline: Annotated[str, "Deadline in format like 'September 30' or '30.10'."],
    ) -> str:
        """Add a new task with priority and deadline (stored as DD-MM)."""
        deadline_str = deadline  # Default to raw input
        try:
            # Try parsing "September 30" or "30.10"
            if "." in deadline:  # Format like 30.10
                parsed_date = datetime.strptime(deadline, "%d-%m")
            else:  # Format like September 30
                parsed_date = datetime.strptime(deadline, "%B %d")
            deadline_str = parsed_date.strftime("%d-%m")
        except ValueError:
            # If parsing fails, keep raw input
            pass

        self.tasks.append({"name": name, "priority": priority, "deadline": deadline_str})
        return f"Task '{name}' added with priority {priority} and deadline {deadline_str}."

    def list_tasks(self) -> Annotated[List[Dict[str, str]], "Returns all tasks with details."]:
        """List all tasks."""
        return self.tasks if self.tasks else "No tasks available."

    def filter_by_priority(
        self,
        priority: Annotated[str, "Priority level to filter: high, medium, low."],
    ) -> List[Dict[str, str]]:
        """Return tasks matching the given priority."""
        filtered = [task for task in self.tasks if task["priority"] == priority]
        return filtered if filtered else f"No tasks with priority {priority}."


# Create agent with tools
tools = TaskManagerTools()

agent = client.create_agent(
    instructions="You are a helpful and precise assistant that manages tasks with priorities and deadlines.",
    tools=[tools.add_task, tools.list_tasks, tools.filter_by_priority]
)

# Multi-turn conversation
thread = agent.get_new_thread()
user_messages = [
    "Add a task to prepare workshop slides, high priority by 30.10",
    "Send invites by december 30th",
    "I need to send an email to Kate by nov 4",
    "List all tasks.",
    "Show me tasks with low priority."
]

for m in user_messages:
    print("*** User:", m)
    response = await agent.run(m, thread=thread)
    print("*** Agent:", response.text)

<details>
    <summary>See the solution</summary>

```python
from typing import Annotated, List, Dict
from datetime import datetime


class TaskManagerTools:
    def __init__(self):
        self.tasks: List[Dict[str, str]] = []  # expected list of dicts, each task: {"name": str, "priority": str, "deadline": str}

    def add_task(
        self,
        name: Annotated[str, "The task description."],
        priority: Annotated[str, "Priority level: high, medium, low."],
        deadline: Annotated[str, "Deadline in format like 'September 30' or '30.10'."],
    ) -> str:
        """Add a new task with priority and deadline (stored as DD-MM)."""
        deadline_str = deadline  # Default to raw input
        try:
            # Try parsing "September 30" or "30.10"
            if "." in deadline:  # Format like 30.10
                parsed_date = datetime.strptime(deadline, "%d-%m")
            else:  # Format like September 30
                parsed_date = datetime.strptime(deadline, "%B %d")
            deadline_str = parsed_date.strftime("%d-%m")
        except ValueError:
            # If parsing fails, keep raw input
            pass

        self.tasks.append({"name": name, "priority": priority, "deadline": deadline_str})
        return f"Task '{name}' added with priority {priority} and deadline {deadline_str}."

    def list_tasks(self) -> Annotated[List[Dict[str, str]], "Returns all tasks with details."]:
        """List all tasks."""
        return self.tasks if self.tasks else "No tasks available."

    def filter_by_priority(
        self,
        priority: Annotated[str, "Priority level to filter: high, medium, low."],
    ) -> List[Dict[str, str]]:
        """Return tasks matching the given priority."""
        filtered = [task for task in self.tasks if task["priority"] == priority]
        return filtered if filtered else f"No tasks with priority {priority}."


# Create agent with tools
tools = TaskManagerTools()

agent = client.create_agent(
    instructions="You are a helpful and precise assistant that manages tasks with priorities and deadlines.",
    tools=[tools.add_task, tools.list_tasks, tools.filter_by_priority]
)

# Multi-turn conversation
thread = agent.get_new_thread()
user_messages = [
    "Add a task to prepare workshop slides, high priority by 30.10",
    "Send invites by december 30th",
    "I need to send an email to Kate by nov 4",
    "List all tasks.",
    "Show me tasks with low priority."
]

for m in user_messages:
    print("*** User:", m)
    response = await agent.run(m, thread=thread)
    print("*** Agent:", response.text)
```
</details>

### Agent Middleware

Middleware in the Agent Framework provides a powerful way to intercept, modify, and enhance agent interactions at various stages of execution. You can use middleware to implement cross-cutting concerns such as logging, security validation, error handling, and result transformation without modifying your core agent or function logic.

Function-based middleware is the simplest way to implement middleware using async functions. This approach is ideal for stateless operations and provides a lightweight solution for common middleware scenarios.

1. Agent Middleware
2. Function Middleware
3. Chat Middleware

The following example demonstrates how middleware can be used to log execution details at a function (logs when a tool function is called and completed) and agent (logs before and after the entire agent run) levels. This gives you visibility and control over the agent’s behavior.

In [ ]:
from datetime import datetime
from agent_framework import FunctionInvocationContext
from typing import Callable, Awaitable
from agent_framework import agent_middleware

def get_time():
    """Get the current time."""
    return datetime.now().strftime("%H:%M:%S")

# middleware hooks onto function invocation events
async def logging_function_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """Middleware that logs function execution."""
    print(f"[Function] Calling {context.function.name}")

    await next(context) # executes the actual function or any next MW

    print(f"[Function] {context.function.name} completed")
    print(f"[Function] Result: {context.result}")


# agent-level MW is applied to the entire agent run
@agent_middleware  # Explicitly marks as agent middleware
async def logging_agent_middleware(context, next):    # don't need to explicitly declare type annotations
    """Agent middleware with decorator - types are inferred."""
    print(f"[Agent] BEFORE execution")
    await next(context)     # executes the agent logic
    print(f"[Agent] AFTER execution | Agent output: {context.result}")

agent = client.create_agent(
    name="TimeAgent",
    instructions="You can tell the current time.",
    tools=[get_time],    
    middleware=[logging_function_middleware, logging_agent_middleware]
)

result = await agent.run("What time is it?")
print("Final response: ", result.text)

print("----------------------------------")

# the following response doesn't invoke get_time() so logging_function MW won't fire
result = await agent.run(
    "Can you help?"
)

print("Final response: ", result.text)

### Example: Controlling Light States with an Agent
This example shows how an agent can perform actions on light states directly using a custom tool class. We define a `LightsTools` class that manages a list of lights, each represented by a `LightModel` typed dictionary. 

The agent can:
- List all lights and their current states.
- Get the state of a specific light by ID.
- Change the state (on/off, brightness, color) of a light.

By exposing these methods as tools, the agent can interact with and modify the environment dynamically.

In [ ]:
from typing import TypedDict, Annotated, List, Optional

# Define a typed dictionary for light properties
class LightModel(TypedDict):
    id: int
    name: str
    is_on: bool | None
    brightness: int | None
    hex: str | None

# Tool class for managing lights
class LightsTools:
    def __init__(self, lights: list[LightModel]): # stores a list of lights in self.lights
        self.lights = lights

    def get_lights(self) -> List[LightModel]:                
        """Get a list of lights and their current state."""
        return self.lights

    def get_state(
        self, id: Annotated[int, "The ID of the light"] #  Annotated adds documentation for LLMs
    ) -> Optional[LightModel]:
        """Get the state of a particular light."""
        for light in self.lights:
            if light["id"] == id:
                return light
        return None
    
    def change_state(
        self, id: Annotated[int, "The ID of the light"], new_state: LightModel
    ) -> dict:
        """Change the state of the light and returns previous/current info."""
        for light in self.lights:
            if light["id"] == id:
                previous = light.copy()  # snapshot before change
                light["is_on"] = new_state.get("is_on", light["is_on"])
                light["brightness"] = new_state.get("brightness", light["brightness"])
                light["hex"] = new_state.get("hex", light["hex"])
                return {
                    "previous": previous,
                    "current": light
                }
        return None

Now that we have the `LightsTools` class defined, we need to provide initial data for the lights and create an instance of the class. This instance will hold the current state of all lights and expose methods that the agent can use.

In [ ]:
lights = [
    {"id": 1, "name": "Table Lamp", "is_on": False, "brightness": 100, "hex": "FF0000"},
    {"id": 2, "name": "Porch light", "is_on": False, "brightness": 50, "hex": "00FF00"},
    {"id": 3, "name": "Chandelier", "is_on": True, "brightness": 75, "hex": "0000FF"},
]

# data to instantiate a class and pass methods as tools
tools = LightsTools(lights=lights)

Now that we have defined the `LightsTools` class and instantiated it with initial light data, let's create the agent, attach the tools, and run a command to control the lights.

In [ ]:
agent = client.create_agent(
    instructions="""
    You control and manage smart lights.
    - Check previous states to check if an action needs to be taken
    - When turning lights on or off, do NOT change brightness or hex unless the user explicitly asks.
    - When changing brightness or hex, only modify those values if requested.""",
    
    tools=[tools.get_lights, tools.get_state, tools.change_state]
)

result = await agent.run(
    "turn on all lamps",
    )

print(result)
print("Final data:", tools.lights)

### Exercise - Add Your Own Middleware to an Agent
In the following exercise, we'll be adding middleware for easier debugging and understanding of agent behavior, because currently we can't see anything besides agent's response.
We'll add function middleware which intercepts function calls within agents, chat middleware for OpenAI usage logging and agent-level middleware which is persistent across all runs for tool call counting.

In [ ]:
import logging
import time
from agent_framework import function_middleware, agent_middleware, chat_middleware

# Configure logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", force=True
)
logger = logging.getLogger(__name__)

@agent_middleware       # decorator allows to skip type annotations like AgentRunContext
async def agent_middleware(context,next                           
) -> None:
    start = time.perf_counter()

    logger.info(f"[Agent] Starting execution")

    await next(context)

    # Calculate total duration
    duration = time.perf_counter() - start
    logger.info(f"[Agent] Agent run completed in {duration:.4f}s")

@chat_middleware 
async def chat_middleware(context, next
) -> None:
    logger.info(f"[Chat] Sending {len(context.messages)} messages to AI")

    await next(context)

    logger.info("[Chat] Response received")

@function_middleware        
async def function_middleware(context, next
) -> None:
    logger.info(f"[Function] Calling - {context.function.name} | Args: {context.arguments}")
    start = time.perf_counter()

    await next(context)

    duration = time.perf_counter() - start
    logger.info(f"[Function] Completed \"{context.function.name}\" in {duration:.6f}s | Result: {context.result}")


agent = client.create_agent(
    instructions="""
    You control and manage smart lights.
    - Check previous states to check if an action needs to be taken
    - When turning lights on or off, do NOT change brightness or hex unless the user explicitly asks.
    - When changing brightness or hex, only modify those values if requested.""",
    tools=[tools.get_lights, tools.get_state, tools.change_state],
    middleware=[agent_middleware, chat_middleware, function_middleware]
)

result = await agent.run(
    "turn off all lights and give me their final state"
    )

print("Assistant >" + str(result))

<details>
    <summary>See the solution</summary>

```python
import logging
import time
from agent_framework import function_middleware, agent_middleware, chat_middleware

# Configure logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", force=True
)
logger = logging.getLogger(__name__)

@agent_middleware       # decorator allows to skip type annotations like AgentRunContext
async def agent_middleware(context,next                           
) -> None:
    start = time.perf_counter()

    logger.info(f"[Agent] Starting execution")

    await next(context)

    # Calculate total duration
    duration = time.perf_counter() - start
    logger.info(f"[Agent] Agent run completed in {duration:.4f}s")

@chat_middleware 
async def chat_middleware(context, next
) -> None:
    logger.info(f"[Chat] Sending {len(context.messages)} messages to AI")

    await next(context)

    logger.info("[Chat] Response received")

@function_middleware        
async def function_middleware(context, next
) -> None:
    logger.info(f"[Function] Calling - {context.function.name} | Args: {context.arguments}")
    start = time.perf_counter()

    await next(context)

    duration = time.perf_counter() - start
    logger.info(f"[Function] Completed \"{context.function.name}\" in {duration:.6f}s | Result: {context.result}")


agent = client.create_agent(
    instructions="""
    You control and manage smart lights.
    - Check previous states to check if an action needs to be taken
    - When turning lights on or off, do NOT change brightness or hex unless the user explicitly asks.
    - When changing brightness or hex, only modify those values if requested.""",
    tools=[tools.get_lights, tools.get_state, tools.change_state],
    middleware=[agent_middleware, chat_middleware, function_middleware]
)

result = await agent.run(
    "turn off all lights and give me their final state"
    )

print("Assistant >" + str(result))
```
</details>

### Exercise: Create Your Own Function Tools

Now it's your turn to create a custom plugin and use it with an agent. Your plugin could simulate a database lookup, a translation service, or any other useful functionality.

Your task:
1. Create a custom plugin class with at least two kernel functions
2. Add the plugin to the kernel
3. Create an agent that uses your plugin
4. Test the agent with appropriate queries

In [ ]:
# Example solution
from typing import Annotated

class TranslationTool:
    """A tool that simulates a language translation service."""
    
    # Dictionary of supported languages and their codes
    _supported_languages = {
        "english": "en",
        "spanish": "es",
        "french": "fr",
        "german": "de",
        "italian": "it",
        "japanese": "ja",
        "chinese": "zh",
    }
    
    # Simple translation dictionary for common phrases (in a real plugin, this would use an API)
    _translations = {
        "hello": {
            "es": "hola",
            "fr": "bonjour",
            "de": "hallo",
            "it": "ciao",
            "ja": "こんにちは",
            "zh": "你好"
        },
        "goodbye": {
            "es": "adiós",
            "fr": "au revoir",
            "de": "auf wiedersehen",
            "it": "arrivederci",
            "ja": "さようなら",
            "zh": "再见"
        },
        "thank you": {
            "es": "gracias",
            "fr": "merci",
            "de": "danke",
            "it": "grazie",
            "ja": "ありがとう",
            "zh": "谢谢"
        },
        "please": {
            "es": "por favor",
            "fr": "s'il vous plaît",
            "de": "bitte",
            "it": "per favore",
            "ja": "お願いします",
            "zh": "请"
        },
        "how are you": {
            "es": "¿cómo estás?",
            "fr": "comment allez-vous?",
            "de": "wie geht es dir?",
            "it": "come stai?",
            "ja": "お元気ですか？",
            "zh": "你好吗？"
        }
    }
    
  
    async def list_languages(self) -> str:
        """Return a list of languages supported by the translation service."""
        languages = list(self._supported_languages.keys())
        return f"Supported languages: {', '.join(languages)}"
    

    async def translate(
        self, 
        text: Annotated[str, "The English text to translate."],
        target_language: Annotated[str, "The language to translate to."]
    ) -> str:
        """Translate the given English text to the specified target language."""
        # Convert to lowercase for matching
        text_lower = text.lower()
        language_lower = target_language.lower()
        
        # Check if the target language is supported
        if language_lower not in self._supported_languages:
            return f"Sorry, translation to {target_language} is not supported. Use list_languages() to see supported languages."
        
        # Get the language code
        lang_code = self._supported_languages[language_lower]
        
        # Check if we have a translation for this phrase
        for phrase, translations in self._translations.items():
            if phrase in text_lower:
                if lang_code in translations:
                    # Replace the phrase with its translation
                    translated = text.lower().replace(phrase, translations[lang_code])
                    return f"Translation to {target_language}: {translated}"
        
        # For phrases we don't have stored, we'll simulate a generic response
        return f"Translation to {target_language} would normally be provided through an external API."
    

    async def detect_language(self, text: Annotated[str, "The text to analyze."]) -> str:
        """Detect the likely language of the provided text."""
        text_lower = text.lower()
        
        # Simple detection based on known translations
        for phrase, translations in self._translations.items():
            for lang_code, translated_phrase in translations.items():
                if translated_phrase in text_lower:
                    # Get the language name from the code
                    language = next(name for name, code in self._supported_languages.items() if code == lang_code)
                    return f"Detected language: {language.capitalize()}"
        
        # Default response if no match found
        return "Language detection would normally use detailed analysis through an external API."

tools = TranslationTool()

# Create a language assistant
language_agent = client.create_agent(
    name="LanguageAssistant",
    instructions="""You are a helpful language assistant that can help users with translations.
    Use the available functions to provide translations and language information.
    
    When answering questions:
    1. If a user wants to know what languages are supported, use the list_languages function
    2. If a user wants a translation, use the translate function
    3. If a user provides text in a foreign language, try to identify it with detect_language
    4. Provide cultural context when relevant to the translation
    """,
    tools = [tools.list_languages, tools.translate, tools.detect_language]
)

thread = language_agent.get_new_thread()
    
# Test questions
questions = [
    "What languages can you translate to?",
    "How do I say 'thank you' in French?",
    "Can you detect what language this is: 'Grazie mille'?"
]


for question in questions:
    print(f"\nUser: {question}")
    response = await language_agent.run(question, thread=thread)
    print(f"Language Assistant: {response.text}")

### TODO4 - do weather information as a plugin instead of translation service


<details>
  <summary>Click to see solution</summary>
  
  ```python

from typing import Annotated

class TranslationTool:
    """A tool that simulates a language translation service."""
    
    # Dictionary of supported languages and their codes
    _supported_languages = {
        "english": "en",
        "spanish": "es",
        "french": "fr",
        "german": "de",
        "italian": "it",
        "japanese": "ja",
        "chinese": "zh",
    }
    
    # Simple translation dictionary for common phrases (in a real plugin, this would use an API)
    _translations = {
        "hello": {
            "es": "hola",
            "fr": "bonjour",
            "de": "hallo",
            "it": "ciao",
            "ja": "こんにちは",
            "zh": "你好"
        },
        "goodbye": {
            "es": "adiós",
            "fr": "au revoir",
            "de": "auf wiedersehen",
            "it": "arrivederci",
            "ja": "さようなら",
            "zh": "再见"
        },
        "thank you": {
            "es": "gracias",
            "fr": "merci",
            "de": "danke",
            "it": "grazie",
            "ja": "ありがとう",
            "zh": "谢谢"
        },
        "please": {
            "es": "por favor",
            "fr": "s'il vous plaît",
            "de": "bitte",
            "it": "per favore",
            "ja": "お願いします",
            "zh": "请"
        },
        "how are you": {
            "es": "¿cómo estás?",
            "fr": "comment allez-vous?",
            "de": "wie geht es dir?",
            "it": "come stai?",
            "ja": "お元気ですか？",
            "zh": "你好吗？"
        }
    }
    
  
    async def list_languages(self) -> str:
        """Return a list of languages supported by the translation service."""
        languages = list(self._supported_languages.keys())
        return f"Supported languages: {', '.join(languages)}"
    

    async def translate(
        self, 
        text: Annotated[str, "The English text to translate."],
        target_language: Annotated[str, "The language to translate to."]
    ) -> str:
        """Translate the given English text to the specified target language."""
        # Convert to lowercase for matching
        text_lower = text.lower()
        language_lower = target_language.lower()
        
        # Check if the target language is supported
        if language_lower not in self._supported_languages:
            return f"Sorry, translation to {target_language} is not supported. Use list_languages() to see supported languages."
        
        # Get the language code
        lang_code = self._supported_languages[language_lower]
        
        # Check if we have a translation for this phrase
        for phrase, translations in self._translations.items():
            if phrase in text_lower:
                if lang_code in translations:
                    # Replace the phrase with its translation
                    translated = text.lower().replace(phrase, translations[lang_code])
                    return f"Translation to {target_language}: {translated}"
        
        # For phrases we don't have stored, we'll simulate a generic response
        return f"Translation to {target_language} would normally be provided through an external API."
    

    async def detect_language(self, text: Annotated[str, "The text to analyze."]) -> str:
        """Detect the likely language of the provided text."""
        text_lower = text.lower()
        
        # Simple detection based on known translations
        for phrase, translations in self._translations.items():
            for lang_code, translated_phrase in translations.items():
                if translated_phrase in text_lower:
                    # Get the language name from the code
                    language = next(name for name, code in self._supported_languages.items() if code == lang_code)
                    return f"Detected language: {language.capitalize()}"
        
        # Default response if no match found
        return "Language detection would normally use detailed analysis through an external API."

tools = TranslationTool()

# Create a language assistant
language_agent = client.create_agent(
    name="LanguageAssistant",
    instructions="""You are a helpful language assistant that can help users with translations.
    Use the available functions to provide translations and language information.
    
    When answering questions:
    1. If a user wants to know what languages are supported, use the list_languages function
    2. If a user wants a translation, use the translate function
    3. If a user provides text in a foreign language, try to identify it with detect_language
    4. Provide cultural context when relevant to the translation
    """,
    tools = [tools.list_languages, tools.translate, tools.detect_language]
)

thread = language_agent.get_new_thread()
    
# Test questions
questions = [
    "What languages can you translate to?",
    "How do I say 'thank you' in French?",
    "Can you detect what language this is: 'Grazie mille'?"
]


for question in questions:
    print(f"\nUser: {question}")
    response = await language_agent.run(question, thread=thread)
    print(f"Language Assistant: {response.text}")
```
</details>